# Apprentissage supervisé

Le but de ce TP est d'implémenter et tester les algorithmes vues en classes.

## Utilitaire

Ci-dessous des fonctions à utiliser pour le TP.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np


def plot_points(points, ax, color):
    """
        Plots points.
        :param points: the points to plot
        :type points: iterable
        :param ax: the axis to plot in
        :type ax: matplotlib.axis
        :param color: the plotted points color
        :type color: str
    """
    ax.scatter(points[:, 0], points[:, 1], c=color)

    
def plot_dataset(X, Y, ax, **parameters):
    """
        Plots the dataset.
        :param X: the points to plot
        :type X: np.array
        :param Y: the points classes
        :type Y: np.array
        :param ax: the axis to plot in
        :type ax: matplotlib.axis
        :param colors: the plotted points colors depending on the class
        :type colors: iterable
    """
    for x, color in zip([X[Y==y] for y in set(Y)], parameters['colors']):
        plot_points(x, ax, color)

        
def mesh_from(X, number=100):
    """
        Constructs mesh points from some 2D instances
        :param X: the 2D instances
        :type X: np.array
        :param number: the quatization frequency
        :type number: int
        :return: the mesh points
        :rtype: tuple
    """
    return np.meshgrid(
        np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, number),
        np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, number),
    )


def plot_separator(xx, yy, ax, classifier, **parameters):
    """
        Plots separator.
        
        :param xx: mesh first coordinates
        :type xx: np.array
        :param yy: mesh second coordinates
        :type yy: np.array
        :param ax: subplot to draw in
        :type ax: matplotlib.axis
        :param classifier: the prediction function
        :type classifier: callable
    """
    Z = classifier(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, **parameters)


On génère trois bases de données pour l'apprentissage:

In [ ]:
import sklearn.datasets
import sklearn.decomposition

X_blob, Y_blob = sklearn.datasets.make_blobs(n_samples=178, centers=2, random_state=10, cluster_std=5)
X_circles, Y_circles = sklearn.datasets.make_circles(n_samples=178, factor=.3, noise=.1)
wine_data = sklearn.datasets.load_wine()
X_wine, Y_wine = sklearn.decomposition.PCA(n_components=2).fit_transform(wine_data.data), wine_data.target
datasets = [('blobs', X_blob, Y_blob), ('circles', X_circles, Y_circles), ('wine', X_wine, Y_wine)]

figure, (ax1, ax2, ax3) = plt.subplots(1, 3)
figure.set_figheight(10)
figure.set_figwidth(30)

plot_dataset(X_blob, Y_blob, ax1, colors=['r', 'b'])
ax1.set_title('Simple synthetic dataset')
plot_dataset(X_circles, Y_circles, ax2, colors=['r', 'b'])
ax2.set_title('Circular synthetic dataset')
plot_dataset(X_wine, Y_wine, ax3, colors=['r', 'b', 'g'])
ax3.set_title('First 2 dimensions of dataset')
plt.tight_layout()
plt.show()


## Classifieur de centroïdes

1.
Implémenter la fonction `centroid_train` qui entraîne un modèle de classifieur de centroïdes.

2.
Implémenter le fonction `centroid_test` qui prédit, pour une instance, la classe la plus probable.

3.
Lancer le code et commenter les résultats.

In [ ]:
import operator

# Euclidian Norm
euclidian = lambda x, y: np.linalg.norm(np.array(x) - np.array(y))

def centroid_train(X, Y):
    return [
        # For each class `cls` compute the centroid
        for cls in set(Y)
    ]


def centroid_test(X, centroids, metric=euclidian):
    return np.array(
        [
            # For each observation `x` predict the class
            for x in X
        ]
    )


figure, axes = plt.subplots(1, 3)
figure.set_figheight(10)
figure.set_figwidth(30)

for (name, X, Y), ax, colors in zip(datasets, axes, [['r', 'b'], ['r', 'b'], ['r', 'b', 'g']]):
    plot_dataset(X, Y, ax, colors=colors)
    centroids = centroid_train(X, Y)
    xx, yy = mesh_from(X, 150)
    plot_separator(
        xx,
        yy,
        ax,
        lambda x:  centroid_test(x, centroids),
        cmap=plt.cm.viridis,
        alpha=0.5
    )
    ax.set_title('Centroid classifier applied to' + name)

plt.tight_layout()
plt.show()


## k-NN

4.
Utiliser la classe `KNeighborsClassifier` pour compléter le code suivant. Commenter les résultats.


In [ ]:
import sklearn.neighbors

for k in range(3,12): # Different k values are tried
    figure, axes = plt.subplots(1, 3)
    figure.set_figheight(10)
    figure.set_figwidth(30)
    for (name, X, Y), ax, colors in zip(datasets, axes, [['r', 'b'], ['r', 'b'], ['r', 'b', 'g']]):
        plot_dataset(X, Y, ax, colors=colors)
        xx, yy = mesh_from(X, 150)
        model = None # The classifier model
        plot_separator(
            xx,
            yy,
            ax,
            lambda x: model.predict(x),
            cmap=plt.cm.viridis,
            alpha=0.5
        )
        ax.set_title('k-NN classifier applied to' + name + 'with k=' + str(k))

plt.tight_layout()
plt.show()


## Arbre de décision

5.
Utiliser la classe `DecisionTreeClassifier` pour compléter le code suivant. Commenter les résultats.

In [ ]:
import sklearn.tree

for criterion in ['gini', 'entropy']:
    figure, axes = plt.subplots(1, 3)
    figure.set_figheight(10)
    figure.set_figwidth(30)
    for (name, X, Y), ax, colors in zip(datasets, axes, [['r', 'b'], ['r', 'b'], ['r', 'b', 'g']]):
        plot_dataset(X, Y, ax, colors=colors)
        xx, yy = mesh_from(X, 150)
        model = None # The classifier model
        plot_separator(
            xx,
            yy,
            ax,
            lambda x: model.predict(x),
            cmap=plt.cm.viridis,
            alpha=0.5
        )
        ax.set_title('Decision tree classifier applied to ' + name + ' with '+ criterion + ' criterion ')

plt.tight_layout()
plt.show()


6.
Imposer aux classifieur, comme critère d'arrêt, une profondeur maximale. Commenter les résultats.

In [ ]:
import sklearn.tree

for max_depth in range(1, 10):
    figure, axes = plt.subplots(1, 3)
    figure.set_figheight(10)
    figure.set_figwidth(30)
    for (name, X, Y), ax, colors in zip(datasets, axes, [['r', 'b'], ['r', 'b'], ['r', 'b', 'g']]):
        plot_dataset(X, Y, ax, colors=colors)
        xx, yy = mesh_from(X, 150)
        model = None # The classifier model
        plot_separator(
            xx,
            yy,
            ax,
            lambda x: model.predict(x),
            cmap=plt.cm.viridis,
            alpha=0.5
        )
        ax.set_title('Decision tree classifier applied to ' + name + ' with max depth '+ str(max_depth) + ' criterion ')

plt.tight_layout()
plt.show()


7.
Changer le critère d'arrêt on fixant différent seuil de pureté de feuilles. Commenter.

In [ ]:
import sklearn.tree

for impurity in [l/20 for l in range(1, 10)]:
    figure, axes = plt.subplots(1, 3)
    figure.set_figheight(10)
    figure.set_figwidth(30)
    for (name, X, Y), ax, colors in zip(datasets, axes, [['r', 'b'], ['r', 'b'], ['r', 'b', 'g']]):
        plot_dataset(X, Y, ax, colors=colors)
        xx, yy = mesh_from(X, 150)
        model = None # The classifier model
        plot_separator(
            xx,
            yy,
            ax,
            lambda x: model.predict(x),
            cmap=plt.cm.viridis,
            alpha=0.5
        )
        ax.set_title('Decision tree classifier applied to ' + name + ' with '+ str(impurity) + ' maximum impurity criterion')

plt.tight_layout()
plt.show()


## Forêt aléatoire

8.
Compléter les fonctions `select_attributes`, `random_forest_train` et `random_forest_test`, puis lancer le code suivant. Commenter les résultats.

In [ ]:
import random


def sampling_indices(X):
    # Draw a random number of dimensions
    d = random.randint(1, X.shape[1])
    # return indices to be sampled
    return random.sample(range(X.shape[1]), d)


def majority_vote(trees_predictions):
    return [
        max(
            [
                (cls, sum(tree_predictions == cls))
                for cls in set(tree_predictions)
            ],
            key=operator.itemgetter(-1)
        )[0]
        for tree_predictions in trees_predictions
    ]


def select_attributes(X, trees_sampling_indices):
    return [
        # For each tree index select the attributes in the sampling indices. 
        # Must be a couple (tree, attributes)
        for tree_idx, sampling_indices in trees_sampling_indices
    ]


def random_forest_train(X, Y, number_trees=100, max_depth=4):
    trees_sampling_indices = [
        # For each tree index draw the sampling indices.
        # Must be a couple (tree, indices). Use the function `sampling_indices()`
        for tree_idx in range(number_trees)
    ]
    X_as = select_attributes(X, trees_sampling_indices)
    trained_trees = {
        tree_idx: # fitted tree with max depth `max_depth`
        for (tree_idx, X_a) in X_as
    }
    return (
        trees_sampling_indices,
        trained_trees
    )


def random_forest_test(X, trees_sampling_indices, trees):
    X_as = select_attributes(X, trees_sampling_indices)
    trees_predictions = np.array(
        [
            # For each tree predict the classe of each observation
            for tree_idx, X_a in X_as
        ]
    ).transpose()
    return np.array(
        majority_vote(trees_predictions)
    )


figure, axes = plt.subplots(1, 3)
figure.set_figheight(10)
figure.set_figwidth(30)

for (name, X, Y), ax, colors in zip(datasets, axes, [['r', 'b'], ['r', 'b'], ['r', 'b', 'g']]):
    plot_dataset(X, Y, ax, colors=colors)
    trees_sampling_indices, trees = random_forest_train(X, Y, number_trees=500, max_depth=1)
    xx, yy = mesh_from(X, 150)
    plot_separator(
        xx,
        yy,
        ax,
        lambda x:  random_forest_test(x, trees_sampling_indices, trees),
        cmap=plt.cm.viridis,
        alpha=0.5
    )
    ax.set_title('Random Forest classifier applied to' + name)
    
plt.tight_layout()
plt.show()
